In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql
from get_sequence_urls import get_sequences

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [3]:
f = open("vinyasa_urls", "rb")
vinyasa_urls = pickle.load(f)
f.close()

In [4]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Vinyasa"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [5]:
vinyasa_classes = [get_poses(url) for url in vinyasa_urls[4000:6000]]

In [6]:
df = pd.DataFrame(vinyasa_classes)

In [12]:
f = open("vinyasa_df", "rb")
vinyasa_df = pickle.load(f)
f.close()

In [13]:
vinyasa_df = pd.concat([vinyasa_df, df])

In [14]:
f = open("vinyasa_df", "wb")
pickle.dump(vinyasa_df, f)
f.close()